In [1]:
%load_ext autoreload
%autoreload 2
import gc
from glob import glob
import os
import re
import sys
import yaml
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import timer, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename, logger_func
from ieee_train import eval_train, eval_check_feature
from kaggle_utils import reduce_mem_usage, move_feature
import shutil

In [2]:
test_preds = read_pkl_gzip('../output/fold_test_pred/20190917_1010_Each_Fold__CV0-9454660522014052__feature750.gz')

In [10]:
from sklearn.metrics import roc_auc_score

COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'

pred = \
test_preds[0] * 0.05 + \
test_preds[1] * 0.2  + \
test_preds[2] * 0.1  + \
test_preds[3] * 0.3 + \
test_preds[4] * 0.15  + \
test_preds[5] * 0.2
# pred = np.mean(test_preds, axis=0)
submission = pd.read_csv('../input/sample_submission.csv')
submission.columns = [COLUMN_ID, 'pred']
submission['pred'] = pred

bear = pd.read_csv('../input/20190913_ieee__bear_probing.csv').iloc[:, [0, 3, 4, 6]]
bear = bear[bear[COLUMN_TARGET]==-1]
bear = bear.iloc[:, [0,1,3]]
bear.columns = [COLUMN_ID, 'data_type', COLUMN_TARGET]

bear_score = submission.merge(bear, how='inner', on=COLUMN_ID)
public = bear_score[bear_score['data_type']=='test_public']
private = bear_score[bear_score['data_type']=='test_private']


public_score = roc_auc_score(public[COLUMN_TARGET].values, public['pred'].values)
private_score = roc_auc_score(private[COLUMN_TARGET].values, private['pred'].values)
all_score = roc_auc_score(bear_score[COLUMN_TARGET].values, bear_score['pred'].values)
print(public_score, private_score, all_score)

0.9674430812094064 0.9647794610372981 0.9660911573029863
